# This is the capstone project notebook

In [1]:
import pandas as pd
import numpy as np

print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Create a dataframe from wikipedia that will consist of three columns: PostalCode, Borough, and Neighborhood

In [2]:
# The code was removed by Watson Studio for sharing.

,Postal_Code,Borough,Neighborhood
0,MIJ,SCARBOROUGH,SCARBOROUGH VILLAGE
1,M9W,ETOBICOKE NORTHWEST,"CLAIRVILLE, HUMBERWOOD, WOODBINE DOWNS, WEST H..."
2,M9V,ETOBICOKE,"SOUTH STEELES, SILVERSTONE, HUMBERGATE, JAMEST..."
3,M9R,ETOBICOKE,"KINGVIEW VILLAGE, ST. PHILLIPS, MARTIN GROVE G..."
4,M9P,ETOBICOKE,WESTMOUNT


# Get the latitude and the longitude coordinates of each neighborhood by Postal Code. 

In [3]:
# add latitude and longitude

In [4]:

body = client_83a01ad55ed74e879ad36f9e780d4e6b.get_object(Bucket='financialmarketscustomerofferaffi-donotdelete-pr-0clbhikycqkxw6',Key='Geospatial_Coordinates1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_5 = pd.read_csv(body)
df_data_5.head()


,Postal_Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Merge latitude and longitude dataframe by Postal Code to pair neighborhoods with their latitude and longitude

In [5]:
# Merge Latitude and Longitude dataframe by Postal Code to the neighborhoods and boroughs dataframe
df_data_4 = pd.merge(df_data_3, df_data_5, how='inner', on = 'Postal_Code')
df_data_4.head()

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M9W,ETOBICOKE NORTHWEST,"CLAIRVILLE, HUMBERWOOD, WOODBINE DOWNS, WEST H...",43.706748,-79.594054
1,M9V,ETOBICOKE,"SOUTH STEELES, SILVERSTONE, HUMBERGATE, JAMEST...",43.739416,-79.588437
2,M9R,ETOBICOKE,"KINGVIEW VILLAGE, ST. PHILLIPS, MARTIN GROVE G...",43.688905,-79.554724
3,M9P,ETOBICOKE,WESTMOUNT,43.696319,-79.532242
4,M9N,YORK,WESTON,43.706876,-79.518188


# Explore and cluster the neighborhoods in Toronto - there are 12 boroughs and 102 neighborhoods.

In [6]:
print('The merged dataframe has {} boroughs and {} neighborhoods in Toronto.'.format(
        len(df_data_4['Borough'].unique()),
        df_data_4.shape[0]
    )
)

The merged dataframe has 12 boroughs and 102 neighborhoods in Toronto.


In [7]:
# install map visualizations
!pip install folium
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Libraries imported.


# Get the latitude and longitude of Toronto

In [8]:
# get latitude and longitude of Toronto from geopy
from geopy.geocoders import Nominatim

address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


# Display all Postal Codes in Toronto on a Map

In [9]:
# create map of Toronto using latitude and longitude values that plots all the Postal Codes
map_toronto = folium.Map(location=[43.6534817, -79.3839347], zoom_start=10)

# Add a marker for each postal code
for i in range(0,len(df_data_4)):
   folium.Marker(
      location=[df_data_4.iloc[i]['Latitude'], df_data_4.iloc[i]['Longitude']],
      popup=df_data_4.iloc[i]['Postal_Code'],
   ).add_to(map_toronto)

# Show the map 
map_toronto

# Refine data analysis to only the Borough of 'Downtown Toronto'

In [10]:
# Cluster only the neighborhoods of the borough of Downtown Toronto within the city
downtown_data = df_data_4[df_data_4['Borough'] == 'DOWNTOWN TORONTO'].reset_index(drop=True)
downtown_data.head()

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M6G,DOWNTOWN TORONTO,CHRISTIE,43.669542,-79.422564
1,M5X,DOWNTOWN TORONTO,"FIRST CANADIAN PLACE, UNDERGROUND CITY",43.648429,-79.382280
2,M5W,DOWNTOWN TORONTO,ENCLAVE OF M5E,43.646435,-79.374846
3,M5V,DOWNTOWN TORONTO,"CN TOWER, KING AND SPADINA, RAILWAY LANDS, HAR...",43.628947,-79.394420
4,M5T,DOWNTOWN TORONTO,"KENSINGTON MARKET, CHINATOWN, GRANGE PARK",43.653206,-79.400049


In [11]:
print('The borough of Downtown Toronto has {} neighborhoods.'.format(
        len(downtown_data['Neighborhood'].unique()),
        downtown_data.shape[0]
    )
)

The borough of Downtown Toronto has 18 neighborhoods.


# Display the 18 neighborhoods of 'Downtown Toronto' on a map

In [12]:
# create map of Downtown Toronto Neighborhoods by name using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for i in range(0,len(downtown_data)):
   folium.Marker(
      location=[downtown_data.iloc[i]['Latitude'], downtown_data.iloc[i]['Longitude']],
      popup=downtown_data.iloc[i]['Neighborhood'],
      icon=folium.DivIcon(html=f"""<div style="font-family: arial; font-size: 13px; font-weight: 'bold'; color:blue")>{downtown_data.iloc[i]['Neighborhood']}</div>""")
   ).add_to(map_downtown)
    
map_downtown

In [13]:
# The code was removed by Watson Studio for sharing.

log in to Foursquare API


# Explore the venues in the first neighborhood 'Christie' using Foursquare API

In [14]:
downtown_data.loc[0, 'Neighborhood'] # explore the first neighborhood

'CHRISTIE'

In [15]:
neighborhood_latitude = downtown_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtown_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of CHRISTIE are 43.669542, -79.4225637.


In [16]:
# limit of number of venues returned by Foursquare API
LIMIT = 100 
radius = 500

In [17]:
# request data from Foursquare API to find 100 venues within 500 meter radius of the neighborhood latitude and longitude
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Fiesta Farms,Grocery Store,43.668471,-79.420485
1,Contra Cafe,Café,43.669107,-79.426105
2,Vinny’s Panini,Italian Restaurant,43.670679,-79.426148
3,Starbucks,Coffee Shop,43.671530,-79.421400
4,Scout and Cash Caffe,Café,43.667360,-79.419938


In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

15 venues were returned by Foursquare.


# Expand the venue search for all 18 neighborhoods in 'Downtown Toronto' borough

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
downtown_venues = getNearbyVenues(names=downtown_data['Neighborhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )

CHRISTIE
FIRST CANADIAN PLACE, UNDERGROUND CITY
ENCLAVE OF M5E
CN TOWER, KING AND SPADINA, RAILWAY LANDS, HARBOURFRONT WEST, BATHURST QUAY, SOUTH NIAGARA, ISLAND AIRPORT
KENSINGTON MARKET, CHINATOWN, GRANGE PARK
UNIVERSITY OF TORONTO, HABORD
COMMERCE COURT, VICTORIA HOTEL
TORONTO DOMINION CENTRE, DESIGN EXCHANGE
HARBOURFRONT EAST, UNION STATION, TORONTO ISLANDS
RICHMOND, ADELAIDE, KING
CENTRAL BAY STREET
BERCZY PARK
ST. JAMES TOWN
GARDEN DISTRICT, RYERSON
REGENT PARK, HARBOURFRONT
CHURCH AND WELLESLEY
ST. JAMES TOWN, CABBAGETOWN
ROSEDALE


In [23]:
# check the size of the dataframe
print(downtown_venues.shape)
downtown_venues.head()

(1211, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,CHRISTIE,43.669542,-79.422564,Fiesta Farms,43.668471,-79.420485,Grocery Store
1,CHRISTIE,43.669542,-79.422564,Contra Cafe,43.669107,-79.426105,Café
2,CHRISTIE,43.669542,-79.422564,Vinny’s Panini,43.670679,-79.426148,Italian Restaurant
3,CHRISTIE,43.669542,-79.422564,Starbucks,43.671530,-79.421400,Coffee Shop
4,CHRISTIE,43.669542,-79.422564,Scout and Cash Caffe,43.667360,-79.419938,Café


# Count the number of venues per neighborhood

In [24]:
downtown_venues.groupby('Neighborhood').count() # number of venues that were returned for each neighborhood

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
BERCZY PARK,57,57,57,57,57,57
CENTRAL BAY STREET,68,68,68,68,68,68
CHRISTIE,15,15,15,15,15,15
CHURCH AND WELLESLEY,79,79,79,79,79,79
"CN TOWER, KING AND SPADINA, RAILWAY LANDS, HARBOURFRONT WEST, BATHURST QUAY, SOUTH NIAGARA, ISLAND AIRPORT",17,17,17,17,17,17
"COMMERCE COURT, VICTORIA HOTEL",100,100,100,100,100,100
ENCLAVE OF M5E,98,98,98,98,98,98
"FIRST CANADIAN PLACE, UNDERGROUND CITY",100,100,100,100,100,100
"GARDEN DISTRICT, RYERSON",100,100,100,100,100,100


# Map the categories via one hot encoding

In [25]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
downtown_onehot.shape

(1211, 207)

In [27]:
# number of venues that were returned for each neighborhood
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

downtown_grouped.shape

(18, 207)

# List each of the top 5 most commons venues per neighborhood

In [28]:
# print each neighborhood with the 5 most common venues from Foursquare API
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----BERCZY PARK----
            venue  freq
0          Bakery  0.05
1     Coffee Shop  0.05
2      Restaurant  0.04
3        Pharmacy  0.04
4  Farmers Market  0.04


----CENTRAL BAY STREET----
                 venue  freq
0          Coffee Shop  0.18
1                 Café  0.06
2       Sandwich Place  0.04
3   Italian Restaurant  0.04
4  Japanese Restaurant  0.03


----CHRISTIE----
                venue  freq
0       Grocery Store  0.27
1                Café  0.20
2                Park  0.13
3         Candy Store  0.07
4  Italian Restaurant  0.07


----CHURCH AND WELLESLEY----
                 venue  freq
0          Coffee Shop  0.08
1  Japanese Restaurant  0.06
2     Sushi Restaurant  0.06
3              Gay Bar  0.04
4           Restaurant  0.04


----CN TOWER, KING AND SPADINA, RAILWAY LANDS, HARBOURFRONT WEST, BATHURST QUAY, SOUTH NIAGARA, ISLAND AIRPORT----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3       Coffee Shop 

In [29]:
# sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# Display the top 10 venues overall for each downtown neighborhood

In [30]:
# display top 10 venues overall for each downtown neighborhood in a dataframe

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,BERCZY PARK,Bakery,Coffee Shop,Cheese Shop,Pharmacy,Beer Bar,Restaurant,Farmers Market,Seafood Restaurant,Cocktail Bar,Clothing Store
1,CENTRAL BAY STREET,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Burger Joint,Japanese Restaurant,Restaurant,Bubble Tea Shop,Salad Place,Office
2,CHRISTIE,Grocery Store,Café,Park,Coffee Shop,Nightclub,Italian Restaurant,Restaurant,Candy Store,Baby Store,Airport
3,CHURCH AND WELLESLEY,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Yoga Studio,Hotel,Fast Food Restaurant,Mediterranean Restaurant,Pub
4,"CN TOWER, KING AND SPADINA, RAILWAY LANDS, HAR...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Boat or Ferry,Plane,Rental Car Location,Sculpture Garden,Coffee Shop,Harbor / Marina


# Cluster the 18 Downtown Toronto neighborhoods in 6 clusters

In [31]:
# cluster the 18 Downtown Toronto neighborhoods into 6 clusters
kclusters = 6

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 5, 3, 0, 3, 3, 3, 3, 3], dtype=int32)

In [32]:
# add clustering labels to create a dataframe that includes the clusters and top 10 venues per neighborhood
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_data

# merge manhattan_grouped with downtown_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

,Postal_Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M6G,DOWNTOWN TORONTO,CHRISTIE,43.669542,-79.422564,5,Grocery Store,Café,Park,Coffee Shop,Nightclub,Italian Restaurant,Restaurant,Candy Store,Baby Store,Airport
1,M5X,DOWNTOWN TORONTO,"FIRST CANADIAN PLACE, UNDERGROUND CITY",43.648429,-79.382280,3,Coffee Shop,Café,Hotel,Gym,Restaurant,Japanese Restaurant,American Restaurant,Asian Restaurant,Bakery,Salad Place
2,M5W,DOWNTOWN TORONTO,ENCLAVE OF M5E,43.646435,-79.374846,3,Coffee Shop,Seafood Restaurant,Italian Restaurant,Café,Hotel,Japanese Restaurant,Pub,Restaurant,Bakery,Beer Bar
3,M5V,DOWNTOWN TORONTO,"CN TOWER, KING AND SPADINA, RAILWAY LANDS, HAR...",43.628947,-79.394420,0,Airport Service,Airport Lounge,Airport Terminal,Boutique,Boat or Ferry,Plane,Rental Car Location,Sculpture Garden,Coffee Shop,Harbor / Marina
4,M5T,DOWNTOWN TORONTO,"KENSINGTON MARKET, CHINATOWN, GRANGE PARK",43.653206,-79.400049,1,Café,Bar,Vegetarian / Vegan Restaurant,Coffee Shop,Vietnamese Restaurant,Grocery Store,Bakery,Park,Caribbean Restaurant,Burger Joint


# Create a map to visualize the cluster results

In [33]:
# create map to visualize the clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Identify the neighborhoods in each of the 6 clusters and the related venue types

In [34]:
# identify the venue cluster types that distinguish the first cluster
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[2] + list(range(4, downtown_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"CN TOWER, KING AND SPADINA, RAILWAY LANDS, HAR...",-79.39442,0,Airport Service,Airport Lounge,Airport Terminal,Boutique,Boat or Ferry,Plane,Rental Car Location,Sculpture Garden,Coffee Shop,Harbor / Marina


In [35]:
# identify the venue cluster types that distinguish cluster 2
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[2] + list(range(4, downtown_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"KENSINGTON MARKET, CHINATOWN, GRANGE PARK",-79.400049,1,Café,Bar,Vegetarian / Vegan Restaurant,Coffee Shop,Vietnamese Restaurant,Grocery Store,Bakery,Park,Caribbean Restaurant,Burger Joint
5,"UNIVERSITY OF TORONTO, HABORD",-79.400049,1,Café,Bar,Sandwich Place,Japanese Restaurant,Bookstore,Bakery,Yoga Studio,Restaurant,Beer Bar,Beer Store


In [36]:
# identify the venue cluster types that distinguish cluster 3
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[2] + list(range(4, downtown_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,ROSEDALE,-79.377529,2,Park,Trail,Playground,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [37]:
# identify the venue cluster types that distinguish cluster 4
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[2] + list(range(4, downtown_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"FIRST CANADIAN PLACE, UNDERGROUND CITY",-79.382280,3,Coffee Shop,Café,Hotel,Gym,Restaurant,Japanese Restaurant,American Restaurant,Asian Restaurant,Bakery,Salad Place
2,ENCLAVE OF M5E,-79.374846,3,Coffee Shop,Seafood Restaurant,Italian Restaurant,Café,Hotel,Japanese Restaurant,Pub,Restaurant,Bakery,Beer Bar
6,"COMMERCE COURT, VICTORIA HOTEL",-79.379817,3,Coffee Shop,Restaurant,Hotel,Café,Gym,American Restaurant,Seafood Restaurant,Bakery,Deli / Bodega,Japanese Restaurant
7,"TORONTO DOMINION CENTRE, DESIGN EXCHANGE",-79.381576,3,Coffee Shop,Hotel,Café,Restaurant,Seafood Restaurant,Italian Restaurant,Salad Place,Bakery,Japanese Restaurant,Gastropub
8,"HARBOURFRONT EAST, UNION STATION, TORONTO ISLANDS",-79.381752,3,Coffee Shop,Aquarium,Café,Hotel,Restaurant,Scenic Lookout,Fried Chicken Joint,Brewery,History Museum,Plaza
9,"RICHMOND, ADELAIDE, KING",-79.384568,3,Coffee Shop,Café,Clothing Store,Restaurant,Thai Restaurant,Gym,Hotel,Steakhouse,Burrito Place,Salad Place
10,CENTRAL BAY STREET,-79.387383,3,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Burger Joint,Japanese Restaurant,Restaurant,Bubble Tea Shop,Salad Place,Office
11,BERCZY PARK,-79.373306,3,Bakery,Coffee Shop,Cheese Shop,Pharmacy,Beer Bar,Restaurant,Farmers Market,Seafood Restaurant,Cocktail Bar,Clothing Store
12,ST. JAMES TOWN,-79.375418,3,Café,Coffee Shop,Restaurant,Gastropub,Cosmetics Shop,Cocktail Bar,Clothing Store,Gym,Moroccan Restaurant,Lingerie Store
13,"GARDEN DISTRICT, RYERSON",-79.378937,3,Coffee Shop,Clothing Store,Cosmetics Shop,Bubble Tea Shop,Café,Japanese Restaurant,Hotel,Ramen Restaurant,Bookstore,Middle Eastern Restaurant


In [38]:
# identify the venue cluster types that distinguish cluster 5
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[2] + list(range(4, downtown_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"REGENT PARK, HARBOURFRONT",-79.360636,4,Coffee Shop,Bakery,Park,Pub,Theater,Café,Breakfast Spot,Electronics Store,Bank,Hotel


In [39]:
# identify the venue cluster types that distinguis cluster 6
downtown_merged.loc[downtown_merged['Cluster Labels'] == 5, downtown_merged.columns[[2] + list(range(4, downtown_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,CHRISTIE,-79.422564,5,Grocery Store,Café,Park,Coffee Shop,Nightclub,Italian Restaurant,Restaurant,Candy Store,Baby Store,Airport
